# Find best assets
The objective will be to define whether certain asset classes are more or less suitable for the strategy we propose.We will study several classes: assets, cryptos, currencies. Then we will study different characteristics like Hurst exponent, volatility, beta.

### Content
* All computations
* What assets classes is the best?
* Other optimal criterions

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

from matplotlib import cycler
colors = cycler('color',
                ['#669FEE', '#66EE91', '#9988DD',
                 '#EECC55', '#88BB44', '#FFBBBB'])
plt.rc('figure', facecolor='#313233')
plt.rc('axes', facecolor="#313233", edgecolor='none',
       axisbelow=True, grid=True, prop_cycle=colors,
       labelcolor='gray')
plt.rc('grid', color='474A4A', linestyle='solid')
plt.rc('xtick', color='gray')
plt.rc('ytick', direction='out', color='gray')
plt.rc('legend', facecolor="#313233", edgecolor="#313233")
plt.rc("text", color="#C9C9C9")
plt.rc('figure', facecolor='#313233')

In [ ]:
import numpy as np
import pandas as pd

import yfinance as yf
from hurst import compute_Hc
import seaborn as sns

import ta
import warnings
from tqdm import tqdm

### Functions

In [ ]:
def rsi(val,neutral, window):
  """ 
        ------------------------------------------------------------------------------
        | Output: The function gives the returns of rsi strategy                     |
        ------------------------------------------------------------------------------
        | Inputs: -val (type dataframe pandas): Entry values of the stock            |
        |         -neutral (float): Value of neutrality, i.e. no action zone         |
        |         -window (float): rolling period for rsi                            |
        ------------------------------------------------------------------------------
  """

  # Print Error if there is no column Adj Close in the dataframe
  if "Adj Close" not in val.columns:
    ValueError("We need have a columns name Adj Close because all computation are about this column")
  
  val["rsi"] = ta.momentum.rsiIndicator(f["Adj Close"], window=window).rsi()


  
  """ Long buy Signal """
  # We put threshold
  overbuy = 70
  neutral_buy = 50 + neutral

  # Put nan values for the signal long columns
  val["signal_long"] = np.nan
  val["yersteday_rsi"] = f["rsi"].shift(1)
  # We need define the Open Long signal (rsi yersteday<55 and rsi today>55)
  val.loc[(val["rsi"]>neutral_buy) & (val["yersteday_rsi"]<neutral_buy), "signal_long"] = 1

  # We need define the Close Long signal (rsi yersteday>55 and rsi today<55) False signal
  val.loc[(val["rsi"]<neutral_buy)&(val["yersteday_rsi"]>neutral_buy), "signal_long"] = 0

  # We need define the Close Long signal (rsi yersteday>70 and rsi today<70) Over buy signal
  val.loc[(val["rsi"]<overbuy)&(val["yersteday_rsi"]>overbuy), "signal_long"] = 0



  """Short sell signal """
  # We put threshold
  oversell = 30
  neutral_buy = 50 - neutral

  # Put nan values for the signal short columns
  val["signal_short"] = np.nan
  val["yersteday_rsi"] = val["rsi"].shift(1)
  # We need define the Open Short signal (rsi yersteday>45 and rsi today<45)
  val.loc[(val["rsi"]<neutral_buy) & (val["yersteday_rsi"]>neutral_buy), "signal_short"] = -1

  # We need define the Close Short signal (rsi yersteday<45 and rsi today>45) False signal
  val.loc[(val["rsi"]>neutral_buy)&(val["yersteday_rsi"]<neutral_buy), "signal_short"] = 0

  # We need define the Close Short signal (rsi yersteday<30 and rsi today>30) Over sell signal
  val.loc[(val["rsi"]>oversell)&(val["yersteday_rsi"]<oversell), "signal_short"] = 0


  """ Compute the returns """
  # Compute the percentage of variation of the asset
  val["pct"] = val["Adj Close"].pct_change(1)

  # Compute the positions
  val["Position"] = (val["signal_short"].fillna(method="ffill")+ val["signal_long"].fillna(method="ffill"))

  # Compute the return of the strategy
  val["return"] = val["pct"]*(val["Position"].shift(1))

  return val["return"]

In [ ]:
def beta_function(serie):

  # Get SP500 data
  sp500 = yf.download("^GSPC")[["Adj Close"]].pct_change(1)

  # Change column name
  sp500.columns = ["SP500"]

  # Concatenate
  g = pd.concat((serie,sp500), axis=1)

  # Compute the beta
  beta = np.cov(g[[serie.name, "SP500"]].dropna().values,rowvar=False)[0][1] / np.var(g["SP500"].dropna().values)
  return beta


In [ ]:
def drawdown_function(serie):

  # We compute Cumsum of the returns
  cum = serie.dropna().cumsum()+1

  # We compute max of the cumsum on the period (accumulate max)
  running_max = np.maximum.accumulate(cum)

  # We compute drawdown
  drawdown  = cum/running_max - 1 
  return drawdown

In [ ]:
def BackTest(serie):

  # Import the benchmark
  sp500 = yf.download("^GSPC")["Adj Close"].pct_change(1)
  
  # Change the name
  sp500.name = "SP500"

  # Concat the returns and the sp500
  val = pd.concat((return_serie,sp500), axis=1).dropna()
  
  # Compute the drawdown
  drawdown = drawdown_function(serie)
  
  # Compute max drawdown
  max_drawdown = -np.min(drawdown)




  # Put a subplots
  fig, (cum, dra) = plt.subplots(1,2, figsize=(15,6))
  
  # Put a Suptitle
  fig.suptitle("Backtesting", size=20)

  # Returns cumsum chart
  cum.plot(serie.cumsum(), color="#39B3C7")

  # SP500 cumsum chart
  cum.plot(val["SP500"].cumsum(), color="#B85A0F")

  # Put a legend
  cum.legend(["Portfolio", "SP500"])
  
  # Set individual title
  cum.set_title("Cumulative Return", size=13)

  # Put the drawdown
  dra.fill_between(drawdown.index,0,drawdown, color="#C73954", alpha=0.65)

  # Set individual title
  dra.set_title("Drawdown", size=13)

  # Plot the graph
  plt.show()




  # Compute the sortino
  sortino = np.sqrt(252) * serie.mean()/serie.loc[serie<0].std()

  # Compute the beta
  beta = np.cov(val[["return", "SP500"]].values,rowvar=False)[0][1] / np.var(val["SP500"].values)

  # Compute the alpha
  alpha = 252*serie.mean() - 252*beta*serie.mean()

  # Print the statistics
  print(f"Sortino: {np.round(sortino,3)}")
  print(f"Beta: {np.round(beta,3)}")
  print(f"Alpha: {np.round(alpha,3)}")
  print(f"MaxDrawdown: {np.round(max_drawdown*100,3)} %")

# Section1:  Computations

### Whats Hurst Exponent?

The Hurst exponent is a statistic that helps to understand the behavior of a TimeSeries.

* 0.5<Hurst <1: Trending movement
* 0.5 = Hurst: Random walk
* 0<Hurst<0.5: Antipersitent movement

In [ ]:
# Trending
# Compute Hurst Exponent
arr = np.linspace(0,300,150)+100
hurst = compute_Hc(arr)[0]

# Show the results
plt.plot(arr)
plt.title(f"{hurst}")
plt.show()

In [ ]:
# Antipersistent
# Compute Hurst Exponent
arr = np.cos(np.linspace(0,300,150))
hurst = compute_Hc(arr)[0]

# Show the results
plt.plot(arr)
plt.title(f"{hurst}")
plt.show()

In [ ]:
# Random
# Compute Hurst Exponent
np.random.seed(56)
arr = np.cumsum(np.random.randn(150))
hurst = compute_Hc(arr)[0]

# Show the results
plt.plot(arr)
plt.title(f"{hurst}")
plt.show()

### Computations

In [ ]:
# We dowload Names.csv because it contains many yahoo ticler
assets = pd.read_csv("/content/Names.csv")["Symbol"]

# Initialize our lists
Statistics = []
col = []

for fin in tqdm(assets):

  # We put try for the asset which have less than 100 values
  try:
    print(fin)
    
    # We download data for each asset
    f = yf.download(fin).dropna()

    # Create a list to put the following statistics
    statistics = list()

    # Compute the Hurst
    statistics.append(compute_Hc(f["Adj Close"])[0])

    # Compute the volatility
    statistics.append(np.sqrt(252)*f["Adj Close"].pct_change(1).dropna().std())

    # Compute the beta
    statistics.append(beta_function(f["Adj Close"].pct_change(1).dropna()))

    # Compute strat return
    statistics.append(rsi(f,5,14).mean()*252)

    # Put statistics list in Statistics to have a list of lists
    Statistics.append(statistics)
    
    # Put column name in the list because some columns dont have 100 values
    col.append(fin)
  
  # If the assets has not 100 values we pass to the next
  except:
    pass


# We create dataframe with all the previous statistics
resume = pd.DataFrame(Statistics, columns=["Hurst", "Volatility", "Beta", "Sum strategy Returns"], index=col)
resume

In [ ]:
# We extract classe of the actif 
clustering = pd.read_csv("/content/Names.csv", index_col="Symbol")
del clustering["Unnamed: 0"]
clustering

In [ ]:
# Concat resume clustering to have the classe of the assets
g = pd.concat((resume,clustering), axis=1).dropna()

# Section 2: What assets classes is the best?

In [ ]:
# We are going to plot the density of the strategy returns by the classe of the assets


# Plot the densities
sns.displot(data=g, x="Sum strategy Returns", kind="kde", hue="dummy")

# Limit the axis
plt.xlim((-1.15,1.15))

# plot the graph
plt.show()

In [ ]:
# Describe by currency
g.loc[g["dummy"]=="Currency"].describe()

In [ ]:
# Describe by Crypto
g.loc[g["dummy"]=="Crypto"].describe()

In [ ]:
# Describe by asset
g.loc[g["dummy"]=="Asset"].describe()

# Section 3: Other optimal criterions

In [ ]:
g.describe()

In [ ]:
# We are going to plot the density of the strategy returns by the HURST
g["Hurst_dum"] = "Low"
g.loc[g["Hurst"]>0.56, "Hurst_dum"] = "High"

# Plot the densities
sns.displot(data=g, x="Sum strategy Returns", kind="kde", hue="Hurst_dum")

# Limit the axis
plt.xlim((-1.15,1.15))

# plot the graph
plt.show()

In [ ]:
# We are going to plot the density of the strategy returns by the classe of the assets
g["Volatility_dum"] = "Low"
g.loc[g["Volatility"]>0.52,"Volatility_dum"] = "High"
# Plot the densities
sns.displot(data=g, x="Sum strategy Returns", kind="kde", hue="Volatility_dum")

# Limit the axis
plt.xlim((-1.15,1.15))

# plot the graph
plt.show()

In [ ]:
# We are going to plot the density of the strategy returns by the classe of the assets
g["Beta_dum"] = "Low"
g.loc[g["Beta"]>1,"Beta_dum"] = "High"
# Plot the densities
sns.displot(data=g, x="Sum strategy Returns", kind="kde", hue="Beta_dum")

# Limit the axis
plt.xlim((-1.15,1.15))

# plot the graph
plt.show()